In [100]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
import time
import os
import re
import pandas as pd
import numpy as np
import urllib
import unicodedata

from urllib.request import urlopen
from urllib.parse import quote_plus

import requests
import shutil
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# from pyvirtualdisplay import Display



In [6]:
spark = SparkSession \
    .builder \
    .appName("multi") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017") \
    .config("spark.mongodb.input.database","ojo_db") \
    .config("spark.mongodb.input.collection", "insta_img") \
    .config("packages org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()
sc =spark.sparkContext

# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument("--single-process")
# chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.add_argument("disable-gpu")
# chrome_options.add_argument("lang=ko_KR") # 한국어!
# chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36")
path='/home/ubuntu/chromedriver'
driver = webdriver.Chrome(path, chrome_options=chrome_options)


/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:13: DeprecationWarning: use options instead of chrome_options


In [110]:
def insta_searching(word):
    url = 'https://www.instagram.com/' + str(word)
    return url

def select_first(driver):
    first=driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
    
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 사진
    try:
        imgUrl = soup.select_one('div.PdwC2 .KL4Bh').img['src']
    except:
        imgUrl = 'not-image'
    # 본문내용
    try:
        content_origin = soup.select('div.C4VMK > span')[0].text
        content = unicodedata.normalize('NFC', content_origin)
    except:
        content = ' '
    #해시태그    
    tags = re.findall(r'#[^\s#,\\]+', content)
    #작성일자   
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    unix = iso2unix(date)
    
    #좋아요
    try:
        like = soup.select('a.zV_Nj')[0].text[4:-1]
    except:
        like = 0
    #위치
    try:
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
        
    data=[content, date, like, place, tags, unix, imgUrl]
    return data

#함수 정의 : 첫번째 게시물 클릭 후 다음 게시물 클릭
def move_next(driver):
    right = driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)
    
# helpers
import time

import pytz

from iso8601 import parse_date
from pytz import reference

UTC = pytz.timezone('UTC')
LOCALZONE = reference.LocalTimezone()

def iso2unix(timestamp):
    """
    Convert a UTC timestamp formatted in ISO 8601 into a UNIX timestamp
    """

    # use iso8601.parse_date to convert the timestamp into a datetime object.
    # set the timezone to the local timezone in order for the time module to
    # properly manipulate it with localtime() and gmtime()
    parsed = parse_date(timestamp).astimezone(LOCALZONE)

    # now grab a time tuple that we can feed mktime
    timetuple = parsed.timetuple()

    # return a unix timestamp
    return time.mktime(timetuple)

def main():
    path='../TIL/Data-Analysis/chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get("https://www.instagram.com/")
    time.sleep(2) #로딩 대기

    driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys('01087334255')
    driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys('hope4255^^')
    driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()

    word= input("검색어 입력 : ")
    word = str(word)
    url = insta_searching(word)
    driver.get(url)
    time.sleep(5)

    select_first(driver)

    results = []
    n=0

    target = 5
    for i in range(target):
        #get content and append in result arr
        data=get_content(driver)
    #     db.insta_img.insert_one({'unix':data[5],'imgUrl':data[6]})
        results.append(data)
        #save data
    #     url = data[-1]
    #     unix = data[-2]
    #     resp = requests.get(url, stream=True)
    #     local_file = open('./img/' + word+'/'+ str(unix) + '.jpg', 'wb')
    #     resp.raw.decode_content = True
    #     shutil.copyfileobj(resp.raw, local_file)
    #     del resp

        #move next
        move_next(driver)

    driver.close()

    results_df = pd.DataFrame(results)
    results_df.columns = ['content', 'date', 'like', 'place', 'tags', 'unix', 'imgUrl']
    # results_df.to_csv('./img/'+ word + '/' + word + '.csv', encoding='utf-8-sig')
    print(results_df)
    
if __name__=='__main__':
    main()

검색어 입력 :  sookso.diary


IndexError: list index out of range

In [108]:
results_df

,content,date,like,place,tags,unix,imgUrl
0,🏷프라이빗하게 즐길 수 있는 한옥숙소 #스테이비투- - - - - - - - - -...,2021-09-18,14,안동,"[#스테이비투-, #숙소다녀왔습니다_안동📍경북]",1.631923e+09,https://scontent-ssn1-1.cdninstagram.com/v/t51...
1,🏷 리버뷰 마운틴뷰룰 원없이 감상할 수 있는 #피그멜리온이펙트- - - - - - ...,2021-09-17,24,피그멜리온이펙트,"[#피그멜리온이펙트-, #숙소다녀왔습니다_춘천📍강원]",1.631837e+09,https://scontent-ssn1-1.cdninstagram.com/v/t51...
2,🏷 제주 동쪽 구좌읍 감성숙소 #올더버드- - - - - - - - - - - - ...,2021-09-16,57,Jeju,"[#올더버드-, #숙소다녀왔습니다_제주📍제주시]",1.631750e+09,https://scontent-ssn1-1.cdninstagram.com/v/t51...
3,🏷 작은 유럽마을에서의 하루 #환타콜라리조트- - - - - - - - - - - ...,2021-09-15,42,환타콜라리조트,"[#환타콜라리조트-, #숙소다녀왔습니다_가평📍경기도]",1.631664e+09,https://scontent-ssn1-1.cdninstagram.com/v/t51...
4,🏷 한국속 가우디 감성 #다우당- - - - - - - - - - - - - - -...,2021-09-14,111,Jeju,"[#다우당-, #숙소다녀왔습니다_제주📍제주]",1.631578e+09,https://scontent-ssn1-1.cdninstagram.com/v/t51...


In [59]:
# import os
# from glob import glob
# # os.getcwd()
# glob('*')
# print("join(): " + os.path.join(os.path.abspath(os.path.dirname('/')), "file.py"))

join(): C:\file.py


In [49]:
# 이미지는 src로 가져와야됨

#인스타그램 주소로 웹 오픈 




# driver.implicitly_wait(5)

# tt = driver.find_element_by_css_selector("html")
# driver.get_screenshot_as_file('capture.png')
# print(tt.text)


# path='/home/ubuntu/chromedriver'
# driver = webdriver.Chrome(path)

# driver.get('https://www.instagram.com/')
# time.sleep(5)

# id_box = driver.find_element_by_css_selector("#loginForm > div > div:nth-child(1) > div > label > input")
# password_box = driver.find_element_by_css_selector("#loginForm > div > div:nth-child(2) > div > label > input")
# login_button = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(3) > button')

#액션체인 정의 act = ActionChains(dr)
# act = ActionChains(driver)

# act.send_keys_to_element(id_box,'01053153004').send_keys_to_element(password_box,'7030136a').click(login_button).perform()
# time.sleep(5)

# 인스타그램 로그인을 위한 계정 정보
# email = '01053153004'
# input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
# input_id.clear()
# input_id.send_keys(email)

# password = '7030136a'
# input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]

# input_pw.clear()
# input_pw.send_keys(password)
# input_pw.submit()


# time.sleep(5)


# 게시물을 조회할 검색 키워드 입력 요청

# if os.path.isdir('./img/'+ word) == False:
#     os.mkdir('./img/'+ word)
url = insta_searching(word)

# 검색 결과 페이지 열기
#driver.get(url)
driver.get(url)
time.sleep(5)

#첫 번째 게시물 클릭

# tt = driver.find_element_by_css_selector("html")

# print(tt.text)

# select_first(driver)

# results = []


# html = driver.page_source
# soup = BeautifulSoup(html)

검색어 입력 :  hi.stay.tour


In [8]:
# # pymongo connect
# client = MongoClient('localhost',27017) # mongodb 27017 port
# db = client.ojo

n=0

target = 500
for i in range(target):
    #try:
    #get content and append in result arr
    data=get_content(driver)
    db.insta_img.insert_one({'unix':data[5],'imgUrl':data[6]})
    results.append(data)
    #save data
    url = data[-1]
    unix = data[-2]
    resp = requests.get(url, stream=True)
    local_file = open('./img/' + word+'/'+ str(unix) + '.jpg', 'wb')
    resp.raw.decode_content = True
    shutil.copyfileobj(resp.raw, local_file)
    del resp

    #move next
    move_next(driver)
    #except:
        #time.sleep(2)
        #move_next(driver)
        
'''
    # This is the image url.
    image_url = imglist[n]
    # Open the url image, set stream to True, this will return the stream content.
    resp = requests.get(image_url, stream=True)
    # Open a local file with wb ( write binary ) permission.
    
    local_file = open('./img/' + urllist[i] + str(n) + '.jpg', 'wb')
    
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    resp.raw.decode_content = True
    # Copy the response stream raw data to local image file.
    shutil.copyfileobj(resp.raw, local_file)
    # Remove the image url response object.
    n +=1
    del resp
'''
driver.close()

검색어 입력 : hi.stay.tour


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div._9AhH0"}
  (Session info: chrome=93.0.4577.82)


In [ ]:
results_df = pd.DataFrame(res
                         )
results_df.columns = ['content', 'date', 'like', 'place', 'tags', 'unix', 'imgUrl']
results_df.to_csv('./img/'+ word + '/' + word + '.csv', encoding='utf-8-sig')


In [7]:
input_pw

<selenium.webdriver.remote.webelement.WebElement (session="084341e53a53f0fd7b6260a2e2726b87", element="df14bff3-0720-4d2a-b1db-9bd783975f37")>

In [8]:
input_id

<selenium.webdriver.remote.webelement.WebElement (session="084341e53a53f0fd7b6260a2e2726b87", element="087b2da9-82d6-40ee-9b39-1e3fc8b3d40e")>